In [1]:
import numpy as np
import pickle
import glob

from keras.utils import to_categorical
from kf_impl import EKF_CV_MODEL
import pdb

Using TensorFlow backend.


### Data Loading

In [2]:
# The function to extract training data from pickles
def extract_data(pklfile, crop=None):
    with open(pklfile, 'rb') as f:
        dataset_all = pickle.load(f)
        
    # All the history trajectoreis (x, y, heading), with shape (batch_size, sequence_length, feature_dims)
    history_traj_data = np.array(dataset_all['features'])[:, :, :]

    # All the future trajectoreis (x, y, heading), with shape (batch_size, sequence_length, feature_dims)
    future_traj_data = np.array(dataset_all['labels'])[:, :, :-1]

    # All the goal positins and occupancy (x, y, occup), with shape (batch_size, (goal_nums * feature_dims))
    goals_position = np.array(dataset_all['goals'])
    goals_position = goals_position.reshape((goals_position.shape[0], goals_position.shape[1] * goals_position.shape[2]))

    # All intention labels, with shape (batch_size, goal_nums)
    goal_idx = np.array(dataset_all['labels'])[:, 0, -1]
    # Convert to one-hot and the last one is undecided (-1)
    one_hot_goal = to_categorical(goal_idx, num_classes=33)
    
    if crop is not None:
        raise NotImplementedError("TODO")
    
    return history_traj_data, future_traj_data, goals_position, one_hot_goal

In [3]:
def build_train_test_splits(pklfiles_to_process, num_pkl_folds=5):
    # Using a suboptimal approach here:
    # Just build a list of dictionaries, where entry_i 
    # corresponds to train split i.

    train_sets = []
    test_sets  = []
    
    
    inds = np.arange(num_pkl_folds)
    
    for hold_out_ind in inds:
        train_inds = np.delete(inds, hold_out_ind)
        
        train_set = {"history_traj_data" : None,
                     "future_traj_data"  : None,
                     "goal_position"     : None,
                     "one_hot_goal"      : None}

        test_set  = {"history_traj_data" : None,
                     "future_traj_data"  : None,
                     "goal_position"     : None,
                     "one_hot_goal"      : None}
        
        # Train Set
        for fold in train_inds:
            search_str = 'fold_%d' % fold
            search_res = np.argwhere([search_str in f for f in pklfiles_to_process])
            assert(len(search_res) == 1)
            file_num = search_res[0][0]
            history_traj_data, future_traj_data, goals_position, one_hot_goal = extract_data(pklfiles_to_process[file_num])

            if train_set['history_traj_data'] is None:
                train_set['history_traj_data'] = history_traj_data
                train_set['future_traj_data']  = future_traj_data
                train_set['goal_position']     = goals_position
                train_set['one_hot_goal']      = one_hot_goal         
            else:
                train_set['history_traj_data'] = np.append(train_set['history_traj_data'], history_traj_data, axis=0)
                train_set['future_traj_data']  = np.append(train_set['future_traj_data'], future_traj_data, axis=0)
                train_set['goal_position']     = np.append(train_set['goal_position'], goals_position, axis=0)
                train_set['one_hot_goal']      = np.append(train_set['one_hot_goal'], one_hot_goal, axis=0)

        # Test Set
        for fold in [hold_out_ind]:
            search_str = 'fold_%d' % fold
            search_res = np.argwhere([search_str in f for f in pklfiles_to_process])
            assert(len(search_res) == 1)
            file_num = search_res[0][0]
            history_traj_data, future_traj_data, goals_position, one_hot_goal = extract_data(pklfiles_to_process[file_num])

            if test_set['history_traj_data'] is None:
                test_set['history_traj_data'] = history_traj_data
                test_set['future_traj_data']  = future_traj_data
                test_set['goal_position']     = goals_position
                test_set['one_hot_goal']      = one_hot_goal         
            else:
                test_set['history_traj_data'] = np.append(test_set['history_traj_data'], history_traj_data, axis=0)
                test_set['future_traj_data']  = np.append(test_set['future_traj_data'], future_traj_data, axis=0)
                test_set['goal_position']     = np.append(test_set['goal_position'], goals_position, axis=0)
                test_set['one_hot_goal']      = np.append(test_set['one_hot_goal'], one_hot_goal, axis=0)
                
        train_sets.append(train_set)
        test_sets.append(test_set)
        
    return train_sets, test_sets

In [4]:
# Construct the evaluation datasets.
pklfiles_to_process = glob.glob('./dataset/*.pkl')
pklfiles_to_process.sort()
train_sets, test_sets = build_train_test_splits(pklfiles_to_process, num_pkl_folds=5)

# Build the model bank.
models = [] #TODO, LSTM + EKF
model = EKF_CV_MODEL(x_init=np.zeros(5),
                     P_init=np.eye(5),
                     R=np.diag([1e-3]*3),
                     dt=0.1)

metrics = []

for train_set, test_set in zip(train_sets, test_sets):
    model.fit(train_set, test_set)
    goal_pred, traj_pred = model.predict(test_set)
    model.save()
    metrics.append(evaluate_model(model))

    # TODO: compute metrics and store in an array for final results.
    # keras/sklearn metrics used for common evaluation.

In [5]:
# TODO: port Xu's visualization code.